In [1]:
def parse_data_filename(dataset_name):

SyntaxError: incomplete input (3487064545.py, line 1)

In [2]:
from pathlib import Path

In [56]:
import pandas as pd
from TSB_AD.model_wrapper import run_Unsupervise_AD
from TSB_AD.evaluation.metrics import get_metrics

# Specify Anomaly Detector to use and data directory
AD_Name =  'IForest'   # It can be replaced with any anomaly detector availale in TSB-AD
data_filename = "289_TODS_id_3_Synthetic_tr_500_1st_26.csv" 
data_direc = f'./Datasets/TSB-AD-U/{data_filename}'

bench = pd.read_csv(Path("/Users/shelleygoel/Code/01_statistical_mod_blog/TSB-AD/benchmark_exp/benchmark_eval_results/uni_mergedTable_VUS-PR.csv"))
data_bench = bench[bench["file"] == data_filename]

# Loading Data
df = pd.read_csv(data_direc).dropna()
data = df.iloc[:, 0:-1].values.astype(float)
label = df['Label'].astype(int).to_numpy()

print("Loaded data with shape:", data.shape)
print("Loaded labels with shape:", label.shape)

# Applying Anomaly Detector
output = run_Unsupervise_AD(AD_Name, data)

# Evaluation
evaluation_result = get_metrics(output, label)

print("Evaluation results:\n", evaluation_result)

Loaded data with shape: (5000, 1)
Loaded labels with shape: (5000,)


/Users/shelleygoel/miniconda3/envs/TSB-AD/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



Evaluation results:
 {'AUC-PR': 0.04039756229041421, 'AUC-ROC': 0.4643378708551483, 'VUS-PR': 0.5817781816321658, 'VUS-ROC': 0.9288174253572509, 'Standard-F1': 0.08616958930378206, 'PA-F1': 0.1893491124260355, 'Event-based-F1': 0.08617041738795914, 'R-based-F1': 0.11126689074863513, 'Affiliation-F': 0.6724951159268401}


# Decision scores from Iforest

In [58]:
output.shape

(5000,)

In [59]:
output

array([-0.02142653, -0.02142653, -0.02142653, ...,  0.00416236,
        0.00416236,  0.00416236])

In [ ]:
import plotly

# Performance Metrics
## VUS-PR

In [55]:
data_bench

,file,Sub-IForest,IForest,Sub-LOF,LOF,POLY,MatrixProfile,KShapeAD,SAND,Series2Graph,...,TimesFM,MOMENT (ZS),MOMENT (FT),ts_len,anomaly_len,num_anomaly,avg_anomaly_len,anomaly_ratio,point_anomaly,seq_anomaly
163,289_TODS_id_3_Synthetic_tr_500_1st_26.csv,0.686718,0.605554,0.687278,0.613622,0.56386,0.757794,0.757864,0.712652,0.715515,...,0.93935,0.625988,0.665456,5000,225,123,1.829268,0.045,1,1


In [52]:
print(evaluation_result['VUS-PR'], data_bench['Sub-PCA'].iloc[0])

0.5929659442972613 0.5932984304999823


In [5]:
bench.columns

Index(['file', 'Sub-IForest', 'IForest', 'Sub-LOF', 'LOF', 'POLY',
       'MatrixProfile', 'KShapeAD', 'SAND', 'Series2Graph', 'SR', 'Sub-PCA',
       'Sub-HBOS', 'Sub-OCSVM', 'Sub-MCD', 'Sub-KNN', 'KMeansAD',
       'AutoEncoder', 'CNN', 'LSTMAD', 'TranAD', 'AnomalyTransformer',
       'OmniAnomaly', 'USAD', 'Donut', 'TimesNet', 'FITS', 'OFA', 'Lag-Llama',
       'Chronos', 'TimesFM', 'MOMENT (ZS)', 'MOMENT (FT)', 'ts_len',
       'anomaly_len', 'num_anomaly', 'avg_anomaly_len', 'anomaly_ratio',
       'point_anomaly', 'seq_anomaly'],
      dtype='str')

In [45]:
data_bench[['Sub-IForest', 'IForest', 'Sub-LOF', 'LOF', 'POLY',
       'MatrixProfile', 'KShapeAD', 'SAND', 'Series2Graph', 'SR', 'Sub-PCA',
       'Sub-HBOS', 'Sub-OCSVM', 'Sub-MCD', 'Sub-KNN', 'KMeansAD',
       'AutoEncoder', 'CNN', 'LSTMAD', 'TranAD', 'AnomalyTransformer',
       'OmniAnomaly', 'USAD', 'Donut', 'TimesNet', 'FITS', 'OFA', 'Lag-Llama',
        'MOMENT (ZS)', 'MOMENT (FT)']].idxmax(axis=1)

163    Lag-Llama
dtype: str

In [47]:
data_bench['Lag-Llama']

163    0.774044
Name: Lag-Llama, dtype: float64

In [6]:
df.shape

(5000, 2)

In [7]:
df.head()

,Data,Label
0,-0.027743,0
1,0.128052,0
2,0.154136,0
3,0.259330,0
4,0.391742,0


In [8]:
df[df['Label'] == 1]

,Data,Label
26,0.114275,1
61,-0.524631,1
77,-0.110289,1
78,-0.148179,1
104,0.248437,1
...,...,...
4859,-0.451428,1
4870,-0.347430,1
4872,-0.248660,1
4883,-0.421993,1


# Visualize Anomalies
- for each anomaly location plot points +/- 20


In [9]:
df['anomaly_group'] = (df['Label'] != df['Label'].shift(fill_value=0)).cumsum() * df['Label']

In [10]:
df['anomaly_group'] = (df['Label'].diff().fillna(0) > 0).cumsum() * df['Label']

In [11]:
df[df['Label'] == 1]

,Data,Label,anomaly_group
26,0.114275,1,1
61,-0.524631,1,2
77,-0.110289,1,3
78,-0.148179,1,3
104,0.248437,1,4
...,...,...,...
4859,-0.451428,1,119
4870,-0.347430,1,120
4872,-0.248660,1,121
4883,-0.421993,1,122


In [12]:
df.head()

,Data,Label,anomaly_group
0,-0.027743,0,0
1,0.128052,0,0
2,0.154136,0,0
3,0.259330,0,0
4,0.391742,0,0


In [31]:
window_size = 50
anomaly_group = 80

anomaly_index = df[df['anomaly_group'] == anomaly_group].index
df_to_plot = df.iloc[max(anomaly_index.start - window_size, 0):min(anomaly_index.stop+window_size
                                                                   , df.shape[0])].copy()
df_to_plot

df_to_plot['Label'] = df_to_plot['Label'].astype('category')
df_to_plot['time'] = df_to_plot.index
import plotly.express as px
px.scatter(df_to_plot, x="time", y="Data", color="Label")

In [ ]:
## 

In [26]:
df.groupby('anomaly_group')['Label'].sum().sort_values()

anomaly_group
0       0
89      1
85      1
84      1
83      1
       ..
80     10
104    10
73     10
51     10
88     10
Name: Label, Length: 124, dtype: int64